<a href="https://colab.research.google.com/github/FernandoBRdgz/diplomado_ds_mod4/blob/main/4.6%20Modelaci%C3%B3n%20supervisada%20y%20no%20supervisada%20de%20Big%20Data/4.6.4%20Regresi%C3%B3n%20Log%C3%ADstica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Regresión Logística

In [ ]:
generation = "mod4gen<x>"

In [ ]:
data = spark.read.csv(f"/mnt/{generation}/input/titanic.csv", inferSchema=True, header=True)

In [ ]:
data.count(), len(data.columns)

In [ ]:
data.printSchema()

In [ ]:
data.display()

**EDA**

In [ ]:
data.groupBy("Survived").count().display()

In [ ]:
data.groupBy("Survived").count().display()

In [ ]:
data.select("Sex").display()

In [ ]:
data.summary("count", "mean", "min", "10%", "20%", "30%", "40%", "50%", "60%", "70%", "80%", "90%", "max").select("summary", "Age", "Fare").display()

In [ ]:
data.select("Age").display()

In [ ]:
data.select("Fare").display()

**Pre-procesamiento**

In [ ]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer

In [ ]:
data.columns

In [ ]:
my_cols = data.select(["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"])

In [ ]:
my_final_data = my_cols.na.drop()

In [ ]:
gender_indexer = StringIndexer(inputCol="Sex", outputCol="SexIndex")
gender_encoder = OneHotEncoder(inputCol="SexIndex", outputCol="SexVec")

In [ ]:
embark_indexer = StringIndexer(inputCol="Embarked", outputCol="EmbarkIndex")
embark_encoder = OneHotEncoder(inputCol="EmbarkIndex", outputCol="EmbarkVec")

In [ ]:
assembler = VectorAssembler(inputCols=["Pclass", "SexVec", "Age", "SibSp", "Parch", "Fare", "EmbarkVec"], outputCol="features")

**Modelación**

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression

In [ ]:
log_reg = LogisticRegression(featuresCol="features", labelCol="Survived")

In [ ]:
pipeline = Pipeline(stages=[gender_indexer,
                            embark_indexer,
                            gender_encoder,
                            embark_encoder,
                            assembler,
                            log_reg])

In [ ]:
train_titanic_data, val_titanic_data = my_final_data.randomSplit([0.8, 0.2])

In [ ]:
fit_model = pipeline.fit(train_titanic_data)

In [ ]:
type(fit_model)

In [ ]:
results = fit_model.transform(val_titanic_data)

In [ ]:
results.display()

**Evaluación**

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="Survived")

In [ ]:
type(my_eval)

In [ ]:
results.select("Survived", "prediction").display()

In [ ]:
my_eval.getMetricName()

In [ ]:
auc = my_eval.evaluate(dataset=results)
auc

**Persistencia de modelo**

In [ ]:
fit_model.save(f"/mnt/{generation}/output/lr_model/")

In [ ]:
from pyspark.ml import PipelineModel

In [ ]:
loaded_model = PipelineModel.load(f"/mnt/{generation}/output/lr_model/")

In [ ]:
new_results = loaded_model.transform(val_titanic_data)

In [ ]:
new_results.display()